In [91]:
from implicit.evaluation import  *
from implicit.als import AlternatingLeastSquares as ALS
from implicit.bpr import BayesianPersonalizedRanking as BPR
import numpy as np
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from sklearn.utils import shuffle
from scipy.sparse import *
# import rec_util
pd.options.display.float_format = '{:.2f}'.format


In [2]:
import pandas as pd
raw_tr = pd.read_json("./raw_data/train.json")
raw_te = pd.read_json("./raw_data/val.json")

In [3]:
ret = []
for tag in raw_tr.tags.tolist():
    ret += tag
from collections import Counter
r = dict(Counter(ret))

In [4]:
r = sorted(r.items(), key=lambda x: -x[1])

In [5]:
top_tags = [x[0] for x in r[:1000]]

In [6]:
train_songs = raw_tr['songs'].tolist()
test_songs = raw_te['songs'].apply(lambda x : [float(i)for i in x]).tolist()
train_tags = raw_tr['tags'].tolist()
test_tags = raw_te['tags'].tolist()
test_ids = raw_te['id'].tolist()

In [7]:
from itertools import groupby
tr = []
iid_to_idx = {}
idx = 0

for i, l in enumerate(train_songs):
    view = l
    for item_id in view:
        if item_id not in iid_to_idx:
            iid_to_idx[item_id] = idx
            idx += 1
    view = [iid_to_idx[x] for x in view]
    tr.append(view)

In [16]:
idx = 0
n_items = 615142
tag_to_idx = {} 
for i, tags in enumerate(train_tags):
    for tag in tags:
        if tag not in tag_to_idx:
            tag_to_idx[tag] = n_items + idx
            idx += 1
    tr[i].extend([tag_to_idx[x] for x in tags])
n_items = len(iid_to_idx)
n_tags = len(tag_to_idx)

In [101]:
from itertools import groupby
te = []

idx = 0
for i, l in enumerate(test_songs):
    view = l
    ret = [] 
    for item_id in view:
        if item_id not in iid_to_idx:
            continue
        ret.append(iid_to_idx[item_id])
    te.append(ret)
idx = 0
for i, tags in enumerate(test_tags):
    ret = []
    for tag in tags:
        if tag not in tag_to_idx:
            continue
        ret.append(tag)
    te[i].extend([tag_to_idx[x] for x in ret])

In [20]:
tr = shuffle(tr)

In [21]:
idx_to_iid = {x:y for(y,x) in iid_to_idx.items()}
idx_to_tag = {(x - n_items):y for(y,x) in tag_to_idx.items()}

# sparse matrix
- 방법에 대해 고민 해봐야 함

In [22]:
def boolean_indexing(v):
    lens = np.array([len(item) for item in v])
    mask = lens[:,None] > np.arange(lens.max())
    out = np.zeros(mask.shape,dtype=int)
    out[mask] = np.concatenate(v)
    return out

In [23]:
tr_array = boolean_indexing(tr)

In [24]:
te_array = boolean_indexing(te)

In [78]:
tr_array = np.zeros([len(tr),len(max(tr,key = lambda x: len(x)))])
for i,j in enumerate(tr):
    tr_array[i][0:len(j)] = j

In [ ]:
# 노래 곡수 + 태그의 최대 숫자가 240인데 이렇게 나오면 안되고, 'n_tags + n_items'인 115071 x 644302인 어레이가 나와야 함

In [84]:
np.array([len (i) for i in tr]).max()

240

In [100]:
tr_array.astype(int)[0]

array([548271, 548272, 191482, 548273, 548274, 548275, 548276, 548277,
       548278, 548279, 548280, 548281, 548282,  73077, 477958, 548283,
       490260, 548284,     87,    572,     33,      1,    173,     45,
           20,   1840, 115158, 115643, 115104, 115072, 115244, 115116,
       115091, 116911, 615229, 615714, 615175, 615143, 615315, 615187,
       615162, 616982, 615229, 615714, 615175, 615143, 615315, 615187,
       615162, 616982,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [62]:
te_array.shape

(23015, 105)

In [64]:
n_tags + n_items

644302

In [135]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(tokenizer=lambda x: x, lowercase=False)
tr_array = cv.fit_transform(tr)

# To transform to dense matrix
tr_array.todense()

matrix([[0, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [112]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(tokenizer=lambda x: x, lowercase=False)
te_array = cv.fit_transform(te)

# To transform to dense matrix
te_array.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

# lil_to_csr 이라는 매서드를 사용하는 방법을 고민해야 함

In [115]:
tr_csr = csr_matrix(tr_array, (len(tr), n_tags + n_items))
te_csr = csr_matrix(te_array, (len(te), n_tags + n_items))

In [116]:
import scipy.sparse
r = scipy.sparse.vstack([te_csr, tr_csr])
r = csr_matrix(r)

In [117]:
als_model = ALS(factors=128, regularization=0.08)
als_model.fit(r.T * 15.0)

In [118]:
item_model = ALS(use_gpu=False)
tag_model = ALS(use_gpu=False)
item_model.user_factors = als_model.user_factors
tag_model.user_factors = als_model.user_factors

In [119]:
item_model.item_factors = als_model.item_factors[:n_items]
tag_model.item_factors = als_model.item_factors[n_items:]

In [120]:
als_model.item_factors.shape

(644302, 128)

In [121]:
als_model.item_factors[n_items:]

array([[-2.7139655e-01,  8.1092104e-02, -2.3679076e-02, ...,
        -4.6856251e-01,  5.2570064e-02,  2.4029304e-01],
       [-4.5710164e-01, -7.0614681e-02,  7.0186621e-03, ...,
         3.4151590e-01,  6.6628605e-01,  2.1835957e-01],
       [-4.2948008e-01, -1.3847241e-02, -5.4426279e-02, ...,
         2.7176657e-01,  6.3023746e-01,  2.0602919e-01],
       ...,
       [ 3.5432735e-03,  2.6923401e-04,  1.0970696e-03, ...,
         4.5748334e-03,  3.5547253e-03,  2.3294524e-03],
       [-1.9738213e-03, -6.2862844e-03, -2.1205323e-03, ...,
        -2.0258485e-03,  8.1458660e-03,  2.8332591e-03],
       [-2.4964832e-04, -8.6847757e-04, -2.2691030e-04, ...,
        -4.4478368e-04, -4.3195429e-05, -5.5013713e-04]], dtype=float32)

In [123]:
item_rec_csr = tr_csr[:, :n_items]
tag_rec_csr = tr_csr[:, n_items:]

# test

In [130]:
item_ret = []
tag_ret = []
from tqdm.auto import tqdm
for u in tqdm(range(te_csr.shape[0])):
    item_rec = item_model.recommend(u, item_rec_csr, N=100)
    item_rec = [idx_to_iid[x[0]] for x in item_rec]
    tag_rec = tag_model.recommend(u, tag_rec_csr, N=100)
    tag_rec = [idx_to_tag[x[0]] for x in tag_rec if x[0] in idx_to_tag]
    item_ret.append(item_rec)
    tag_ret.append(tag_rec)

In [132]:
tag_model.item_factors

array([[-2.7139655e-01,  8.1092104e-02, -2.3679076e-02, ...,
        -4.6856251e-01,  5.2570064e-02,  2.4029304e-01],
       [-4.5710164e-01, -7.0614681e-02,  7.0186621e-03, ...,
         3.4151590e-01,  6.6628605e-01,  2.1835957e-01],
       [-4.2948008e-01, -1.3847241e-02, -5.4426279e-02, ...,
         2.7176657e-01,  6.3023746e-01,  2.0602919e-01],
       ...,
       [ 3.5432735e-03,  2.6923401e-04,  1.0970696e-03, ...,
         4.5748334e-03,  3.5547253e-03,  2.3294524e-03],
       [-1.9738213e-03, -6.2862844e-03, -2.1205323e-03, ...,
        -2.0258485e-03,  8.1458660e-03,  2.8332591e-03],
       [-2.4964832e-04, -8.6847757e-04, -2.2691030e-04, ...,
        -4.4478368e-04, -4.3195429e-05, -5.5013713e-04]], dtype=float32)

In [26]:
tag_model.item_factors

array([[ 3.55746090e-01, -5.69548428e-01, -5.59754260e-02, ...,
         4.83457536e-01,  1.95368275e-01,  1.14752978e-01],
       [ 1.13077201e-01,  4.99642581e-01,  1.01401411e-01, ...,
         8.70525062e-01,  5.08861303e-01,  4.67959046e-01],
       [ 1.34080434e-02,  5.13347864e-01,  1.56607270e-01, ...,
         6.01837337e-01,  4.08071131e-01,  3.32719356e-01],
       ...,
       [-5.10524551e-04,  3.28546972e-04,  8.81741638e-04, ...,
         8.72330857e-04,  1.68862971e-04,  1.18788111e-03],
       [ 9.88858286e-04,  1.02399255e-03,  3.29685328e-03, ...,
         1.42479164e-03, -1.69632386e-03, -1.65981124e-03],
       [ 9.73270435e-05,  1.62653669e-04, -2.30579590e-05, ...,
         1.98975016e-04,  2.20649337e-04, -1.06205756e-04]], dtype=float32)

In [133]:
returnval = []
for _id, rec, tag_rec in zip(test_ids, item_ret, tag_ret):
    returnval.append({
        "id": _id,
        "songs": rec[:100],
        "tags": tag_rec[:10]
    })

In [134]:
import json
with open('ret.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(returnval, ensure_ascii=False))